### 1. Set environment parameters

Be sure to set `env_name` to the name of the Unity environment file you want to launch. Ensure that the environment build is in the `python/` directory.

In [1]:
env_name = "./env/env"  # Name of the Unity environment binary to launch
train_mode = True  # Whether to run the environment in training or inference mode

### 2. Load dependencies

The following loads the necessary dependencies and checks the Python version (at runtime). ML-Agents (v0.3 onwards) requires Python 3.

In [2]:
import numpy as np
import scipy.io
import sys

from unityagents import UnityEnvironment

%matplotlib inline

print("Python version:")
print(sys.version)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents (v0.3 onwards) requires Python 3")

Python version:
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


### 3. Start the environment
`UnityEnvironment` launches and begins communication with the environment when instantiated.

Environments contain _brains_ which are responsible for deciding the actions of their associated _agents_. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
env = UnityEnvironment(file_name=env_name)

# Examine environment parameters
print(str(env))

# Set the default brain to work with
default_brain = env.brain_names[0]
brain = env.brains[default_brain]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: Brain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 30
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 1
        Vector Action descriptions: 


Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: Brain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 30
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 1
        Vector Action descriptions: 


## Data Info


Each dataset consists of the data as follows 
- position_x 
- position_y
- velocity_x
- velocity_y
- (1/mass)
- (1/scale)]

### 4. Get data from the environment to make dataset

In [4]:
def get_state(env):
    state = np.zeros([6, 6])
    vector_obs = env.vector_observations[0]
    vector_obs_reshape = np.reshape(vector_obs, (5, 6))
    vector_obs_trans = np.transpose(vector_obs_reshape)
    state[:, :-1] = vector_obs_trans
    # 1/Mass and 1/scale of wall is 0
    state[4, -1] = 0
    state[5, -1] = 0
    
    return state

data_x = []
data_y = []

num_episode = 100

for episode in range(num_episode):
    env_info = env.reset(train_mode=train_mode)[default_brain]
    done = False
    step = 0
    
    state = get_state(env_info)
    
    while True:
        # Random action 
        env_info = env.step(np.random.randint(0, brain.vector_action_space_size, 
                                                  size=(len(env_info.agents))))[default_brain]
        done = env_info.local_done[0]
        
        if done:
            break
        
        next_state = get_state(env_info)
        
        data_x.append(state)
        data_y.append(next_state)
        
        state = next_state
    
    
    print("Episode " + str(episode) + ' / ' + str(num_episode) )
    print("Data length: " + str(len(data_x)))
    
X = np.asarray(data_x)
Y = np.asarray(data_y)

print(X.shape)
print(Y.shape)

data_dict = {'X': X, 'Y': Y}

scipy.io.savemat('./dataset.mat', data_dict)

print('Data is saved!!!')

Episode 0 / 100
Data length: 1000
Episode 1 / 100
Data length: 2000
Episode 2 / 100
Data length: 3000
Episode 3 / 100
Data length: 4000
Episode 4 / 100
Data length: 5000
Episode 5 / 100
Data length: 6000
Episode 6 / 100
Data length: 7000
Episode 7 / 100
Data length: 8000
Episode 8 / 100
Data length: 9000
Episode 9 / 100
Data length: 10000
Episode 10 / 100
Data length: 11000
Episode 11 / 100
Data length: 12000
Episode 12 / 100
Data length: 13000
Episode 13 / 100
Data length: 14000
Episode 14 / 100
Data length: 15000
Episode 15 / 100
Data length: 16000
Episode 16 / 100
Data length: 17000
Episode 17 / 100
Data length: 18000
Episode 18 / 100
Data length: 19000
Episode 19 / 100
Data length: 20000
Episode 20 / 100
Data length: 21000
Episode 21 / 100
Data length: 22000
Episode 22 / 100
Data length: 23000
Episode 23 / 100
Data length: 24000
Episode 24 / 100
Data length: 25000
Episode 25 / 100
Data length: 26000
Episode 26 / 100
Data length: 27000
Episode 27 / 100
Data length: 28000
Episode 28 

In [5]:
env.close()

## Normalization and concatenation

In [7]:
# dataset_train1 = scipy.io.loadmat('./dataset_train1.mat')
# dataset_train2 = scipy.io.loadmat('./dataset_train2.mat')
# dataset_train3 = scipy.io.loadmat('./dataset_train3.mat')
# dataset_train4 = scipy.io.loadmat('./dataset_train4.mat')

# dataset_test = scipy.io.loadmat('./dataset_test.mat')

# data_1x = dataset_train1['X']
# data_1y = dataset_train1['Y']

# data_2x = dataset_train2['X']
# data_2y = dataset_train2['Y']

# data_3x = dataset_train3['X']
# data_3y = dataset_train3['Y']

# data_4x = dataset_train4['X']
# data_4y = dataset_train4['Y']

# data_x = np.concatenate([data_1x, data_2x, data_3x, data_4x], axis = 0)
# data_y = np.concatenate([data_1y, data_2y, data_3y, data_4y], axis = 0)

# data_x_test = dataset_test['X']
# data_y_test = dataset_test['Y']

# normalization_factor = [1.0/5, 1.0/5, 1.0/20, 1.0/20, 1.0, 1.0]

# for i in range(len(normalization_factor)):
#     data_x[:,i,:] = data_x[:,i,:] * normalization_factor[i]
#     data_y[:,i,:] = data_y[:,i,:] * normalization_factor[i]
    
#     data_x_test[:,i,:] = data_x_test[:,i,:] * normalization_factor[i]
#     data_y_test[:,i,:] = data_y_test[:,i,:] * normalization_factor[i]
    
# Train_data_dict = {'X': data_x, 'Y': data_y}
# Test_data_dict = {'X': data_x_test, 'Y': data_y_test}

# scipy.io.savemat('./Training_dataset.mat', Train_data_dict)
# scipy.io.savemat('./Testing_dataset.mat', Test_data_dict)